# Why stock market volatility benefits the wealthy

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import json
import math
import matplotlib.pyplot as plt
import init_objects
from functions.helpers import organise_data, hypothetical_series, get_specific_bootstraps_moments, confidence_interval
from functions.inequality import gini
from model import *
import statsmodels.api as sm
from matplotlib import style
from functions.evolutionaryalgo import *
import scipy.stats as stats
from SALib.sample import latin

In [2]:
style.use('seaborn-ticks')

In [3]:
params = {"trader_sample_size": 10, # selected for comp efficiency
          "n_traders": 1000, # selected for comp efficiency
          "init_stocks": 3788 / float(1000), # Campbell et al. 2019 total stocks / n-agents 
          "ticks": 1000, # lenght of Campbell et al. 2019 data
          "fundamental_value": 818, # average of Campbell et al. 2019 market cap
          "std_fundamental": 0.05, # standard deviation Campell et al. prices, assuming efficient markets TODO
          "base_risk_aversion": 0.7, # estimate from Kim & Lee (2012)
          'spread_max': 0.004087, # estimate from Riordan & Storkenmaier (2012)
          "horizon": int(1000 * 0.35), # estimate based on average churn ratio found by Cella, Ellul and Giannetti (2013)
          "std_noise": 0.01, 
          "w_random": 1.0, 
          # parameter only used for experiment
          "mean_reversion": 0.0,
          # fixed / not modelled parameters
          "fundamentalist_horizon_multiplier": 1.0,
          "strat_share_chartists": 0.0,
          "mutation_intensity": 0.0,
          "average_learning_ability": 0.0,
          "trades_per_tick": 1
         }

## Estimate params to replicate inequality increase?

What are the moments to replicate?

Params to estimate is stdev noise? 

## Run model

In [4]:
NRUNS = 2

In [5]:
trdrs = []
orbs = []
for seed in range(NRUNS):
    traders, orderbook = init_objects.init_objects(params, seed)
    traders, orderbook = volatility_inequality_model2(traders, orderbook, params, seed)
    trdrs.append(traders)
    orbs.append(orderbook)

Start of simulation  0


AttributeError: 'TraderParameters' object has no attribute 'horizon'